<a href="https://colab.research.google.com/github/Sohila912/CNN_Data_Augmentation/blob/main/Data_aug1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the zip file
uploaded = files.upload()


Saving cat_dog.zip to cat_dog (1).zip


In [2]:
import zipfile
import os

# Unzipping the uploaded dataset
with zipfile.ZipFile("/content/cat_dog (1).zip", "r") as zip_ref:
    zip_ref.extractall("cat_dog1")

# Check if the dataset is extracted
os.listdir("cat_dog1")


['train_sub', 'testsub']

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [4]:
img = load_img("/content/catordog.jpg")
x = img_to_array(img) # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape) # Reshape to (1, 3, 150, 150)

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
# i =0
# for batch in datagen.flow(x, batch_size=1,
#                             save_to_dir=r"E:\Python\DL\Data_Augmentation\preview",
#                             save_prefix='cat',
#                             save_format='jpeg'):
#         i += 1
#         if i > 30:  # Generate 20 images only
#             break

# Create the directory inside Colab
save_dir = "/content/preview"
os.makedirs(save_dir, exist_ok=True)

# Use the new directory in your datagen
i = 0
for batch in datagen.flow(x, batch_size=1,
                            save_to_dir=save_dir,  # Updated path
                            save_prefix='cat',
                            save_format='jpeg'):
    i += 1
    if i > 30:  # Generate 30 images only
        break


In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [10]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r"/content/cat_dog1/train_sub",  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"/content/cat_dog1/testsub",
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2488 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [11]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=800 // batch_size
)

# Save the trained weights with the correct filename
model.save_weights("first_try.weights.h5")  # Filename now follows the required convention


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 152ms/step - accuracy: 0.6683 - loss: 0.6719 - val_accuracy: 0.6308 - val_loss: 0.6302
Epoch 2/60
  5/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7806 - loss: 0.5141

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6914 - loss: 0.6104 - val_accuracy: 0.6308 - val_loss: 0.6252
Epoch 3/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 30s 107ms/step - accuracy: 0.6861 - loss: 0.5873 - val_accuracy: 0.6615 - val_loss: 0.5616
Epoch 4/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6972 - loss: 0.5502 - val_accuracy: 0.6308 - val_loss: 0.6442
Epoch 5/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 107ms/step - accuracy: 0.7296 - loss: 0.5636 - val_accuracy: 0.6769 - val_loss: 0.5919
Epoch 6/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6887 - loss: 0.5633 - val_accuracy: 0.6462 - val_loss: 0.6171
Epoch 7/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 107ms/step - accuracy: 0.7137 - loss: 0.5457 - val_accuracy: 0.6654 - val_loss: 0.6882
Epoch 8/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7190 - loss: 0.5297 - val_accuracy: 0.7077 - val_loss: 0.5560
Epoch 9/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - accuracy: 0.7413 - loss: 0.5377 - val_accurac

In [14]:
model.save("first_try.h5")  # Saves the full model


In [16]:
from tensorflow.keras.models import load_model

model = load_model("/content/first_try.h5")  # Replace with your actual model path


In [20]:
import cv2
import numpy as np

def preprocess_image(image_path, img_size=150):
    img = cv2.imread(image_path)  # Read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (img_size, img_size))  # Resize to (150, 150)
    img = img / 255.0  # Normalize (if you did this during training)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img


In [21]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    prediction = model.predict(img)[0]  # Get prediction
    class_label = "Cat" if prediction < 0.5 else "Dog"  # Adjust threshold if needed
    print(f"Prediction: {class_label} ({prediction[0]:.4f})")


In [24]:
predict_image("/content/5.jpg", model)  # Replace with an actual image path


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: Dog (0.9846)


In [ ]:
# model.fit(
#     train_generator,
#     steps_per_epoch=2000 // batch_size,
#     epochs=70,  # Total epochs
#     initial_epoch=50,  # Start from epoch 50
#     validation_data=validation_generator,
#     validation_steps=800 // batch_size
# )
# # Save the trained weights with the correct filename
# model.save_weights('first_try_updated.weights.h5')  # Filename now follows the required convention